In [56]:
import time

import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

from torchvision import transforms
from torchvision.datasets import CIFAR10

## 网络结构

![](https://miro.medium.com/v2/resize:fit:720/format:webp/1*JEGNYy9rXMj_XN7W1Qjo9g.png)
![](https://img-blog.csdnimg.cn/20200104153325358.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L2p1c3Rfc29ydA==,size_16,color_FFFFFF,t_70)

![](https://img-blog.csdnimg.cn/20200104162456690.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L2p1c3Rfc29ydA==,size_16,color_FFFFFF,t_70)

```
    Input
      |
   Conv Layer
      |
   ReLU
      |
   Conv Layer
      |
      |----------------
      |               |
     ReLU             |
      |               |
     Output          Input
      |               |
      ----------------
           |
         Output
```


In [57]:
def conv3x3(in_channels, out_channels, stride=1):
    # 二维卷积层
    # in_channels：输入特征图的通道数。例如，对于RGB图像，in_channels为3。
    # out_channels：输出特征图的通道数。这个值决定了卷积核的数量，即我们希望提取多少个特征。
    # kernel_size：卷积核的大小，可以是一个整数或一个元组。例如，kernel_size=3表示使用3x3的卷积核。
    # stride：卷积核的步幅，决定卷积核在输入特征图上移动的步长。默认值为1。
    # padding：填充方式，为了保持特征图的尺寸，可以在输入特征图的边缘填充0。padding=1表示在所有边缘填充1个像素。
    # bias：是否添加偏置项。默认值为True。这里不使用偏置项（bias=False），因为后面有批量归一化层。
    return nn.Conv2d(
        in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False
    )


class BasicBlock(nn.Module):
    """
    A basic residual block for ResNet.

    Attributes:
        conv1: First convolutional layer.
        bn1: Batch normalization for the first convolutional layer.
        conv2: Second convolutional layer.
        bn2: Batch normalization for the second convolutional layer.
        shortcut: Shortcut connection to match input and output dimensions.
    """

    expansion: int = 1  # 输出通道数相对于输入通道数的扩展倍数。对于基本块，扩展倍数为1。

    def __init__(self, in_channels, out_channels, stride=1):
        """
        Initializes the basic block.

        Args:
            in_channels: Number of input channels.
            out_channels: Number of output channels.
            stride: Stride for the convolution. Default is 1.
        """
        super(BasicBlock, self).__init__()

        self.conv1 = conv3x3(in_channels, out_channels, stride)
        # 对卷积层的输出进行归一化处理。这有助于加速训练并稳定模型。
        self.bn1 = nn.BatchNorm2d(out_channels) 
        self.conv2 = conv3x3(out_channels, out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.shortcut = nn.Sequential()
        # 在残差块中，如果输入和输出的形状不一致（例如通道数不同或步幅不为1），我们需要通过一个卷积层来调整输入的形状，使其与输出形状一致。
        if stride != 1 or in_channels != out_channels:
            # 如果需要，则定义一个包含1x1卷积层和批量归一化层的顺序容器。
            self.shortcut = nn.Sequential(
                nn.Conv2d(
                    in_channels, out_channels, kernel_size=1, stride=stride, bias=False
                ),
                nn.BatchNorm2d(out_channels),
            )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Defines the computation performed at every call.

        Args:
            x: Input tensor.

        Returns:
            Output tensor.
        """
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


# Example usage
# Define an input tensor with shape (batch_size, in_channels, height, width)
x = torch.randn(1, 64, 32, 32)
# Create a basic block instance
block = BasicBlock(64, 64)
# Forward pass
out = block(x)
print(out.shape)

torch.Size([1, 64, 32, 32])


```
    Input
      |
   1x1 Conv (Reduction)
      |
   BatchNorm
      |
    ReLU
      |
   3x3 Conv
      |
   BatchNorm
      |
    ReLU
      |
   1x1 Conv (Expansion)
      |
   BatchNorm
      |----------------
      |               |
     ReLU             |
      |               |
     Output          Input
      |               |
      ----------------
           |
         Output
```

In [58]:
class Bottleneck(nn.Module):
    """
    A bottleneck residual block for ResNet.

    Attributes:
        conv1: First convolutional layer (1x1).
        bn1: Batch normalization for the first convolutional layer.
        conv2: Second convolutional layer (3x3).
        bn2: Batch normalization for the second convolutional layer.
        conv3: Third convolutional layer (1x1).
        bn3: Batch normalization for the third convolutional layer.
        shortcut: Shortcut connection to match input and output dimensions.
    """

    expansion: int = 4

    def __init__(self, in_channels: int, out_channels: int, stride: int = 1) -> None:
        """
        Initializes the bottleneck block.

        Args:
            in_channels: Number of input channels.
            out_channels: Number of output channels.
            stride: Stride for the convolution. Default is 1.
        """
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(
            in_channels, out_channels, kernel_size=1, stride=1, bias=False
        )
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(
            out_channels,
            out_channels,
            kernel_size=3,
            stride=stride,
            padding=1,
            bias=False,
        )
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.conv3 = nn.Conv2d(
            out_channels,
            out_channels * self.expansion,
            kernel_size=1,
            stride=1,
            bias=False,
        )
        self.bn3 = nn.BatchNorm2d(out_channels * self.expansion)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels * self.expansion:
            self.shortcut = nn.Sequential(
                nn.Conv2d(
                    in_channels,
                    out_channels * self.expansion,
                    kernel_size=1,
                    stride=stride,
                    bias=False,
                ),
                nn.BatchNorm2d(out_channels * self.expansion),
            )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Defines the computation performed at every call.

        Args:
            x: Input tensor.

        Returns:
            Output tensor.
        """
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

In [59]:
class ResNet(nn.Module):
    """
    A ResNet model.

    Attributes:
        in_channels: Number of input channels.
        conv1: Initial convolutional layer.
        bn1: Batch normalization for the initial convolutional layer.
        maxpool: Max pooling layer.
        layer1: First layer of residual blocks.
        layer2: Second layer of residual blocks.
        layer3: Third layer of residual blocks.
        layer4: Fourth layer of residual blocks.
        avgpool: Global average pooling layer.
        fc: Fully connected layer.
    """

    def __init__(
        self, block: type[BasicBlock], num_blocks: list[int], num_classes: int = 1000
    ) -> None:
        """
        Initializes the ResNet model.

        Args:
            block: A residual block.
            num_blocks: A list containing the number of blocks in each layer.
            num_classes: Number of output classes. Default is 1000.
        """
        super(ResNet, self).__init__()
        self.in_channels = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

    def _make_layer(
        self, block: type[BasicBlock], out_channels: int, num_blocks: int, stride: int
    ) -> nn.Sequential:
        """
        Creates a layer of residual blocks.

        Args:
            block: A residual block.
            out_channels: Number of output channels.
            num_blocks: Number of blocks in the layer.
            stride: Stride for the first block.

        Returns:
            A sequential container of residual blocks.
        """
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_channels, out_channels, stride))
            self.in_channels = out_channels * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Defines the computation performed at every call.

        Args:
            x: Input tensor.

        Returns:
            Output tensor.
        """
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

In [60]:
def ResNet18(num_classes: int = 1000) -> ResNet:
    """
    Constructs a ResNet-18 model.

    Args:
        num_classes: Number of output classes. Default is 1000.

    Returns:
        A ResNet-18 model.
    """
    return ResNet(BasicBlock, [2, 2, 2, 2], num_classes)


# Example usage
model = ResNet18(num_classes=10)
x = torch.randn(1, 3, 224, 224)
out = model(x)
print(out.shape)

torch.Size([1, 10])


In [61]:
def ResNet34(num_classes: int = 10) -> ResNet:
    """
    Constructs a ResNet-34 model.

    Args:
        num_classes: Number of output classes. Default is 1000.

    Returns:
        A ResNet-34 model.
    """
    return ResNet(BasicBlock, [3, 4, 6, 3], num_classes)

model = ResNet34(num_classes=10)
x = torch.randn(1, 3, 224, 224)
out = model(x)
print(out.shape)

torch.Size([1, 10])


In [62]:
def ResNet50(num_classes: int = 1000) -> ResNet:
    """
    Constructs a ResNet-50 model.

    Args:
        num_classes: Number of output classes. Default is 1000.

    Returns:
        A ResNet-50 model.
    """
    return ResNet(Bottleneck, [3, 4, 6, 3], num_classes)

model = ResNet50(num_classes=10)
x = torch.randn(1, 3, 224, 224)
out = model(x)
print(out.shape)

torch.Size([1, 10])


In [63]:
def ResNet101(num_classes: int = 1000) -> ResNet:
    """
    Constructs a ResNet-101 model.

    Args:
        num_classes: Number of output classes. Default is 1000.

    Returns:
        A ResNet-101 model.
    """
    return ResNet(Bottleneck, [3, 4, 23, 3], num_classes)

model = ResNet101(num_classes=10)
x = torch.randn(1, 3, 224, 224)
out = model(x)
print(out.shape)

torch.Size([1, 10])


In [65]:
def ResNet152(num_classes: int = 1000) -> ResNet:
    """
    Constructs a ResNet-152 model.

    Args:
        num_classes: Number of output classes. Default is 1000.

    Returns:
        A ResNet-152 model.
    """
    return ResNet(Bottleneck, [3, 8, 36, 3], num_classes)

model = ResNet101(num_classes=10)
x = torch.randn(1, 3, 224, 224)
out = model(x)
print(out.shape)

torch.Size([1, 10])
